In [ ]:
# Importing necessary libraries
import numpy as np
import cv2
import math

In [ ]:
from typing import cast, Optional

In [ ]:
import PIL.Image

def img_scale(img: np.ndarray) -> np.ndarray:
    return (img * 255).astype(np.uint8)

def display_image(img: np.ndarray, _: Optional[str] = None):
    display(PIL.Image.fromarray(img))

def display_contours(img: np.ndarray, contours):
    img_with_contours = img.copy()
    for contour in contours:
        cv2.drawContours(img_with_contours, [contour], 0, (0, 255, 0), 2)
    display_image(img_with_contours)

In [ ]:
img = cv2.imread('Test_Images/temp_ndvi.jpg')
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
col_grey_img = cv2.cvtColor(img_grey, cv2.COLOR_GRAY2RGB) # for printing on top of
display_image(img_grey)

In [ ]:
img_contour_eroded = cv2.erode(
    np.where(cast(np.ndarray, img_grey) > 0, [255], [0]).astype(np.uint8),
    np.ones((5, 5), np.uint8),
    iterations = 3
)

img_eroded_dilated = cv2.dilate(
    img_contour_eroded,
    np.ones((99, 99), np.uint8),
    iterations = 2
)

display_image(img_eroded_dilated)

In [ ]:
initial_contour_mask = img_eroded_dilated

prev_contour_count = math.inf

for _ in range(10): # maximum of 10 recursions
    initial_contours, _ = cv2.findContours(initial_contour_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    initial_contour_mask = np.zeros(img_eroded_dilated.shape, dtype=np.uint8)

    for contour in initial_contours:
        cv2.drawContours(initial_contour_mask, [cv2.convexHull(contour)], 0, (255,), -1)

    display_image(initial_contour_mask)

    if prev_contour_count == len(initial_contours): break
    prev_contour_count = len(initial_contours)
else:
    raise RecursionError()

In [ ]:
chunk_contours, _ = cv2.findContours(initial_contour_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

chunk_masks = []

for contour in chunk_contours:
    contour_mask = np.zeros(initial_contour_mask.shape, dtype=np.uint8)
    cv2.drawContours(contour_mask, [contour], 0, (255,), -1)
    chunk_masks.append(contour_mask)

print(len(chunk_masks))
display_contours(col_grey_img, chunk_contours)

In [ ]:
chunks = []

for mask, contour in zip(chunk_masks, chunk_contours):
    masked_img = cv2.bitwise_or(col_grey_img, col_grey_img, mask=mask)
    x, y, w, h = cv2.boundingRect(contour)
    chunk = masked_img[y:y+h, x:x+w]
    chunks.append(chunk)

for chunk in chunks:
    display_image(chunk)